In [1]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Input, Activation, Dropout, Add, LSTM, GRU, RNN, LayerNormalization, BatchNormalization, Conv1D, MaxPooling1D, Flatten
from keras.optimizers import Adam,SGD
import tensorflow as tf
from keras import Model, regularizers, activations
import pickle
from copy import deepcopy
import random

# disable warnings to ignore overflow error
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [2]:
# parameters for CSTR
T_0_cstr = 300
V_cstr = 1
k_0_cstr = 8.46*(np.power(10,6))
C_p_cstr = 0.231
rho_L_cstr = 1000
Q_s_cstr = 0.0
F_cstr = 5
E_cstr = 5*(np.power(10,4))
delta_H_cstr = -1.15*(np.power(10,4))
R_cstr = 8.314
C_A0s_cstr = 4

t_final_cstr = 0.005
t_step_cstr = 1e-4

# parameters for Batch
V_batch = 1
k_0_batch = 8.46*(np.power(10,7))
C_p_batch = 0.231
rho_L_batch = 1000
Q_s_batch = 0
E_batch = 5*(np.power(10,4))
delta_H_batch = -1.15*(np.power(10,4))
R_batch = 8.314

t_final_batch = 0.05
t_step_batch = 1e-4

# parameters for PFR
k_0_PFR = 8.46 * (np.power(10,6))
C_p_PFR = 0.231
rho_L_PFR = 1000
u_PFR = 2 # volumetric flow rate  (F/A) Superficial velocity
E_by_R_PFR = 5*(np.power(10,4)) / 8.314
delH_term_PFR = -1.15*(np.power(10,4))
U_PFR = 25
Tc_s_PFR = 293
At_PFR = 0.01 # Area for heating rate equation
A_PFR = 0.002  # Area
length_PFR = 1 # total length of reactor
N_PFR = 10     # number of points to discretize the reactor

t_final_PFR = 0.1
t_step_PFR = 0.01

# parameters for neural networks
num_step = 10
num_dims = 4

# parameters for transfer learning
seed = 0
plot = True
rng = np.random.RandomState(seed)
innerepochs = 1 # number of epochs of each inner SGD
niterations = 1500 # number of outer updates; each iteration we sample one task and update on it
ntrain = 32 # size of training minibatches (K)
eval_step = 50 # evaluation step
threshold = 10 # threshold to check data correctness

In [3]:
def generate_new_1000(x):
    return x + x * np.random.uniform(-10, 10)

def generate_new_100(x):
    return x + x * np.random.uniform(-1, 1)

def generate_new_5(x):
    return x + x * np.random.uniform(-0.05, 0.05)

def generate_new_Tc_s(x):
    return x + x * np.random.uniform(-0.068259, 1)

def PFR_simulation(u, delH_term, k_0, C_p, rho_L, E_by_R, Tc, U, At, A, t_final, t_step, init_C, init_T, length, N):

    # Method of lines approximates the spatial derivative using finite difference method which reults in a set of coupled ODE

    def method_of_lines_C(C, T):
        'coupled ODES at each node point'
        D = -u * np.diff(C) / np.diff(z) - k_0 * np.exp(-E_by_R/T[1:]) * C[1:]    # for first order
        return np.concatenate([[0], D]) #C0 is constant at entrance


    def method_of_lines_T(C, T):
        'coupled ODES at each node point'
        D = -u * np.diff(T) / np.diff(z) + (-delH_term/(rho_L*C_p)) * k_0 * np.exp(-E_by_R/T[1:])* C[1:] + (U/(rho_L*C_p*A)) * At * (Tc - T[1:]) # for first order
        return np.concatenate([[0], D]) #T0 is constant at entrance

    z = np.linspace(0, length, N) # discretized length elements

    #initializing arrays
    init_C_A_2_1 = np.zeros(N)
    init_T_2_1 = np.zeros(N)

    init_C_A_2_2 = np.zeros(N)
    init_T_2_2 = np.zeros(N)

    init_C_A_3 = np.zeros(N)
    init_T_3 = np.zeros(N)

    C_A_3 = np.zeros(N)
    T_3 = np.zeros(N)

    dCAdt1 = method_of_lines_C(init_C, init_T)
    dTdt1 = method_of_lines_T(init_C, init_T)

    for i in range(len(init_C)):
        init_C_A_2_1[i] = init_C[i] + dCAdt1[i] * t_step / 2
        init_T_2_1[i] = init_T[i] + dTdt1[i] * t_step / 2

    dCAdt2_1 = method_of_lines_C(init_C_A_2_1, init_T_2_1)
    dTdt2_1 = method_of_lines_T(init_C_A_2_1, init_T_2_1)

    for i in range(len(init_C)):
        init_C_A_2_2[i] = init_C[i] + dCAdt2_1[i] * t_step / 2
        init_T_2_2[i] = init_T[i] + dTdt2_1[i] * t_step / 2

    dCAdt2_2 = method_of_lines_C(init_C_A_2_2, init_T_2_2)
    dTdt2_2 = method_of_lines_T(init_C_A_2_2, init_T_2_2)

    for i in range(len(init_C)):
        init_C_A_3[i] = init_C[i] + dCAdt2_2[i] * t_step / 2
        init_T_3[i] = init_T[i] + dTdt2_2[i] * t_step / 2

    dCAdt3 = method_of_lines_C(init_C_A_3, init_T_3)
    dTdt3 = method_of_lines_T(init_C_A_3, init_T_3)

    dCAdt2 = np.add(dCAdt2_1,dCAdt2_2)
    dCAdt2 = np.divide(dCAdt2,2)

    dTdt2 = np.add(dTdt2_1,dTdt2_2)
    dTdt2 = np.divide(dTdt2,2)

    for i in range(len(init_C)):
        C_A_3[i] = init_C[i] + t_step / 6 * (dCAdt1[i] + 4*dCAdt2[i] + dCAdt3[i])
        T_3[i] = init_T[i] + t_step / 6 * (dTdt1[i] + 4*dTdt2[i] + dTdt3[i])

    return C_A_3 , T_3

def Batch_simulation(V, k_0, E, R, delta_H, rho_L, C_p, Q, t_final, t_step, C_A_initial, T_initial):
    """
        simulating Batch using forward Euler method
    """

    C_A_list = list()  # evolution of CA over time
    T_list = list()  # evolution of T over time

    C_A = C_A_initial
    T = T_initial

    for i in range(int(t_final / t_step)):
        dCAdt = -k_0 * np.exp(-E / (R * T)) * C_A
        dTdt = - delta_H / (rho_L * C_p) * k_0 * np.exp(-E / (R * T)) * C_A + Q / (rho_L * C_p * V)

        T += dTdt * t_step
        C_A += dCAdt * t_step

        if i % 50 == 0:
            C_A_list.append(C_A)
            T_list.append(T)

    return C_A_list, T_list

def CSTR_simulation(F, V, C_A0, k_0, E, R, T_0, delta_H, rho_L, C_p, Q, t_final, t_step, C_A_initial, T_initial):
    """
        simulating CSTR using forward Euler method
    """

    C_A_list = list()  # evolution of CA over time
    T_list = list()  # evolution of T over time

    C_A = C_A_initial
    T = T_initial

    for i in range(int(t_final / t_step)):
        dCAdt = F / V * (C_A0 - C_A) - k_0 * np.exp(-E / (R * T)) * C_A**2
        dTdt = F / V * (T_0 - T) - delta_H / (rho_L * C_p) * k_0 * np.exp(-E / (R * T)) * C_A**2 + Q / (rho_L * C_p * V)

        T += dTdt * t_step
        C_A += dCAdt * t_step

        if i % 5 == 0:
            C_A_list.append(C_A)
            T_list.append(T)

    return C_A_list, T_list

def to_tensor(x):
    return tf.convert_to_tensor(x, dtype=tf.float32)

def train_on_batch(x, y, model, optimizer):
    x = to_tensor(x)
    y = to_tensor(y)

    with tf.GradientTape() as tape:
        YHat = model(x)
        loss = mse_loss_fn(y, YHat)
        grads = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    return loss

def predict(x, model):
    x = to_tensor(x)
    return model(x).numpy()

def compute_loss(x, y, model):
    return np.square(predict(x, model) - y).mean()

In [4]:
def gen_cstr(F, V, C_A0s, k_0, E, R, T_0, delta_H, rho_L, C_p, Q_s, t_final, t_step, num_step, num_dims):
    isCorrect = False
    while isCorrect == False:
        T_0_new = generate_new_100(T_0)
        V_new = generate_new_1000(V)
        F_new = generate_new_1000(F)
        C_A0s_new = generate_new_100(C_A0s)
        Q_s_new = generate_new_1000(Q_s)
        rho_L_new = generate_new_5(rho_L)
        C_p_new = generate_new_5(C_p)
        k_0_new = generate_new_5(k_0)
        E_new = generate_new_5(E)
        delta_H_new = generate_new_5(delta_H)

        # generating inputs and initial states for CSTR
        u1_list = np.linspace(-3.5, 3.5, 4, endpoint=True)
        u2_list = np.linspace(-5e5, 5e5, 4, endpoint=True)
        T_initial = np.linspace(300, 600, 20, endpoint=True)
        CA_initial = np.linspace(0, 6, 20, endpoint=True)

        # restruture the data
        T_start = list()
        CA_start = list()

        for T in T_initial:
            for CA in CA_initial:
                CA_start.append(CA)
                T_start.append(T)

        CA_start = np.array([CA_start])
        T_start = np.array([T_start])
        x_deviation = np.concatenate((CA_start.T, T_start.T), axis=1)

        # get X and y data for training and testing
        CA_output = list()
        T_output = list()
        CA_input = list()
        T_input = list()
        CA0_input = list()
        Q_input = list()

        for u1 in u1_list:
            C_A0 = u1 + C_A0s_new
            for u2 in u2_list:
                Q = u2 + Q_s_new
                for C_A_initial, T_initial in x_deviation:

                    C_A_list, T_list = CSTR_simulation(F_new, V_new, C_A0, k_0_new, E_new, R, T_0_new, delta_H_new, rho_L_new, C_p_new, Q, t_final, t_step, C_A_initial, T_initial)
                    if any(abs(i) < 0.001 for i in T_list) == False and any(abs(i) < 0.001 for i in C_A_list) == False and any(abs(i) > 10000 for i in T_list) == False and any(abs(i) > 10000 for i in C_A_list) == False and any(abs(i) == 0 for i in T_list) == False and any(abs(i) == 0 for i in C_A_list) == False and np.isnan(C_A_list).any() == False and np.isnan(T_list).any() == False and np.isinf(C_A_list).any() == False and np.isinf(T_list).any() == False:
                        CA0_input.append(u1)
                        Q_input.append(u2)
                        CA_input.append(C_A_initial)
                        T_input.append(T_initial)

                        CA_output.append(C_A_list)
                        T_output.append(T_list)

        # regenerate data if requirement is not met
        if len(CA_output) > 5000:

            # collate input for RNN
            CA0_input = np.array(CA0_input)
            CA0_input = CA0_input.reshape(-1,1,1)

            Q_input = np.array(Q_input)
            Q_input = Q_input.reshape(-1,1,1)

            CA_input = np.array(CA_input)
            CA_input = CA_input.reshape(-1,1,1)

            T_input = np.array(T_input)
            T_input = T_input.reshape(-1,1,1)

            RNN_input = np.concatenate((T_input, CA_input, Q_input, CA0_input), axis=2)
            RNN_input = RNN_input.repeat(num_step, axis=1)

            # collate output for RNN
            CA_output = np.array(CA_output)
            CA_output = CA_output.reshape(-1, num_step, 1)

            T_output = np.array(T_output)
            T_output = T_output.reshape(-1, num_step, 1)

            RNN_output = np.concatenate((T_output, CA_output), axis=2)

            # scale the data
            scaler_X = preprocessing.StandardScaler().fit(RNN_input.reshape(-1, num_dims))
            scaler_y = preprocessing.StandardScaler().fit(RNN_output.reshape(-1, 2))

            X = scaler_X.transform(RNN_input.reshape(-1, num_dims))
            y = scaler_y.transform(RNN_output.reshape(-1,2))

            if np.isnan(X).any() == False and np.isnan(y).any() == False and np.isinf(X).any() == False and np.isinf(y).any() == False and any(abs(i) > threshold for i in y.reshape(-1)) == False:
                isCorrect = True

    print("Number of training samples of CSTR: ", int(len(X)/num_step))
    return X.reshape(-1,num_step,num_dims), y.reshape(-1,num_step,2)

In [5]:
def gen_batch(V, k_0, E, R, delta_H, rho_L, C_p, Q_s, t_final, t_step, num_step, num_dims):
    isCorrect = False
    while isCorrect == False:
        V_new = generate_new_1000(V)
        Q_s_new = generate_new_1000(Q_s)
        rho_L_new = generate_new_5(rho_L)
        C_p_new = generate_new_5(C_p)
        k_0_new = generate_new_1000(k_0)
        E_new = generate_new_5(E)
        delta_H_new = generate_new_5(delta_H)

        # generating inputs and initial states for Batch
        u_list = np.linspace(-5e5, 5e5, 4, endpoint=True)
        T_initial = np.linspace(300, 600, 40, endpoint=True)
        CA_initial = np.linspace(0, 6, 40, endpoint=True)

        # restruture the data
        T_start = list()
        CA_start = list()

        for T in T_initial:
            for CA in CA_initial:
                CA_start.append(CA)
                T_start.append(T)

        CA_start = np.array([CA_start])
        T_start = np.array([T_start])
        x_deviation = np.concatenate((CA_start.T, T_start.T), axis=1)

        # get X and y data for training and testing
        CA_output = list()
        T_output = list()
        CA_input = list()
        T_input = list()
        CA0_input = list()
        Q_input = list()

        for u1 in u_list:
            Q = u1 + Q_s_new
            for C_A_initial, T_initial in x_deviation:

                C_A_list, T_list = Batch_simulation(V_new, k_0_new, E_new, R, delta_H_new, rho_L_new, C_p_new, Q, t_final, t_step, C_A_initial, T_initial)
                if any(abs(i) < 0.001 for i in T_list) == False and any(abs(i) < 0.001 for i in C_A_list) == False and any(abs(i) > 10000 for i in T_list) == False and any(abs(i) > 10000 for i in C_A_list) == False and any(abs(i) == 0 for i in T_list) == False and any(abs(i) == 0 for i in C_A_list) == False and np.isnan(C_A_list).any() == False and np.isnan(T_list).any() == False and np.isinf(C_A_list).any() == False and np.isinf(T_list).any() == False:
                    CA0_input.append(0)
                    Q_input.append(u1)
                    CA_input.append(C_A_initial)
                    T_input.append(T_initial)

                    CA_output.append(C_A_list)
                    T_output.append(T_list)

        # regenerate data if requirement is not met
        if len(CA_output) > 5000:

            # collate input for RNN
            CA0_input = np.array(CA0_input)
            CA0_input = CA0_input.reshape(-1,1,1)

            Q_input = np.array(Q_input)
            Q_input = Q_input.reshape(-1,1,1)

            CA_input = np.array(CA_input)
            CA_input = CA_input.reshape(-1,1,1)

            T_input = np.array(T_input)
            T_input = T_input.reshape(-1,1,1)

            RNN_input = np.concatenate((T_input, CA_input, Q_input, Q_input), axis=2)
            RNN_input = RNN_input.repeat(num_step, axis=1)

            # collate output for RNN
            CA_output = np.array(CA_output)
            CA_output = CA_output.reshape(-1, num_step, 1)

            T_output = np.array(T_output)
            T_output = T_output.reshape(-1, num_step, 1)

            RNN_output = np.concatenate((T_output, CA_output), axis=2)

            # scale the data
            scaler_X = preprocessing.StandardScaler().fit(RNN_input.reshape(-1, num_dims))
            scaler_y = preprocessing.StandardScaler().fit(RNN_output.reshape(-1, 2))

            X = scaler_X.transform(RNN_input.reshape(-1, num_dims))
            y = scaler_y.transform(RNN_output.reshape(-1,2))

            if np.isnan(X).any() == False and np.isnan(y).any() == False and np.isinf(X).any() == False and np.isinf(y).any() == False and any(abs(i) > threshold for i in y.reshape(-1)) == False:
                isCorrect = True

    print("Number of training samples of Batch: ", int(len(X)/num_step))
    return X.reshape(-1,num_step,num_dims), y.reshape(-1,num_step,2)

In [6]:
def gen_pfr(u, delH_term, k_0, C_p, rho_L, E_by_R, U, At, A, Tc_s, length, N, t_final, t_step, num_step, num_dims):
    isCorrect = False
    while isCorrect == False:
        u_new = generate_new_100(u)
        U_new = generate_new_100(U)
        A_new = generate_new_1000(A)
        At_new = generate_new_1000(At)
        Tc_s_new = generate_new_Tc_s(Tc_s)
        rho_L_new = generate_new_5(rho_L)
        C_p_new = generate_new_5(C_p)
        k_0_new = generate_new_5(k_0)
        E_by_R_new = generate_new_5(E_by_R)
        delH_term_new = generate_new_5(delH_term)

        # generating inputs and initial states for CSTR, all expressed in deviation form
        u_list = np.linspace(100, 300, 4, endpoint=True)
        T_initial = np.linspace(300, 500, 40, endpoint=True)
        CA_initial = np.linspace(0.5, 3, 40, endpoint=True)

        # restruture the data
        T_start = list()
        CA_start = list()

        for T in T_initial:
            for CA in CA_initial:
                CA_start.append(CA)
                T_start.append(T)

        CA_start = np.array([CA_start])
        T_start = np.array([T_start])
        x_deviation = np.concatenate((CA_start.T, T_start.T), axis=1)

        # get X and y data for physics-informed model
        CA_output = list()
        T_output = list()
        CA_input = list()
        T_input = list()
        Tc_input = list()
        CA0_input = list()

        for u2 in u_list:
            Tc = u2 + Tc_s_new

            for C_A_initial, T_initial in x_deviation:

                z = np.linspace(0, length, N) # discretized length elements

                init_C = np.zeros(N)    # Concentration in reactor at t = 0
                init_C[0] = C_A_initial          # concentration at entrance
                init_T = np.zeros(N)    # T in reactor at t = 0
                for i in range(len(init_T)):
                    if i == 0:
                        init_T[i] = T_initial
                    else:
                        init_T[i] = Tc_s_new

                C_A_list = list()
                T_list = list()

                for i in range(int(t_final / t_step)):

                    CA_next, T_next = PFR_simulation(u_new, delH_term_new, k_0_new, C_p_new, rho_L_new, E_by_R_new, Tc, U_new, At_new, A_new, t_final, t_step, init_C, init_T, length, N)
                    if i % 1 == 0:
                        C_A_list.append(CA_next)
                        T_list.append(T_next)
                    init_C = CA_next
                    init_T = T_next
                if any(abs(i) < 0.001 for i in np.array(T_list)[:,1]) == False and any(abs(i) < 0.001 for i in np.array(C_A_list)[:,1]) == False and any(abs(i) > 10000 for i in np.array(T_list)[:,1]) == False and any(abs(i) > 10000 for i in np.array(C_A_list)[:,1]) == False and any(abs(i) == 0 for i in np.array(T_list)[:,1]) == False and any(abs(i) == 0 for i in np.array(C_A_list)[:,1]) == False and np.isnan(C_A_list).any() == False and np.isnan(T_list).any() == False and np.isinf(C_A_list).any() == False and np.isinf(T_list).any() == False:
                    CA0_input.append(0)
                    Tc_input.append(u2)
                    CA_input.append(C_A_initial)
                    T_input.append(T_initial)

                    CA_output.append(C_A_list)
                    T_output.append(T_list)

        # regenerate data if requirement is not met
        if len(CA_output) > 5000:
            
            CA_output = np.array(CA_output)[:,:,1]
            T_output = np.array(T_output)[:,:,1]

            # collate input for RNN
            CA0_input = np.array(CA0_input)
            CA0_input = CA0_input.reshape(-1,1,1)

            Tc_input = np.array(Tc_input)
            Tc_input = Tc_input.reshape(-1,1,1)

            CA_input = np.array(CA_input)
            CA_input = CA_input.reshape(-1,1,1)

            T_input = np.array(T_input)
            T_input = T_input.reshape(-1,1,1)

            RNN_input = np.concatenate((T_input, CA_input, Tc_input, Tc_input), axis=2)
            RNN_input = RNN_input.repeat(num_step, axis=1)

            # collate output for RNN
            CA_output = np.array(CA_output)
            CA_output = CA_output.reshape(-1, num_step, 1)

            T_output = np.array(T_output)
            T_output = T_output.reshape(-1, num_step, 1)

            RNN_output = np.concatenate((T_output, CA_output), axis=2)

            # scale the data
            scaler_X = preprocessing.StandardScaler().fit(RNN_input.reshape(-1, num_dims))
            scaler_y = preprocessing.StandardScaler().fit(RNN_output.reshape(-1, 2))

            X = scaler_X.transform(RNN_input.reshape(-1, num_dims))
            y = scaler_y.transform(RNN_output.reshape(-1,2))

            if np.isnan(X).any() == False and np.isnan(y).any() == False and np.isinf(X).any() == False and np.isinf(y).any() == False and any(abs(i) > threshold for i in y.reshape(-1)) == False:
                isCorrect = True

    print("Number of training samples of PFR: ", int(len(X)/num_step))
    return X.reshape(-1,num_step,num_dims), y.reshape(-1,num_step,2)

In [7]:
class Model(tf.keras.layers.Layer):

    def __init__(self):
        super(Model, self).__init__()

        self.layer_1 = SimpleRNN(64, activation='relu', return_sequences=True)
        self.layer_2 = SimpleRNN(64, activation='relu', return_sequences=True)
        self.layer_3 = Dense(2, activation='linear')

    def call(self, inputs):
        x = self.layer_1(inputs)
        x = self.layer_2(x)
        x = self.layer_3(x)
        return x

model = Model()

# Necessary to create the model's state.
# The model doesn't have a state until it's called at least once.
_ = model(tf.zeros((ntrain, num_step, num_dims)))

optimizer = tf.keras.optimizers.Adam()
mse_loss_fn = tf.keras.losses.MeanSquaredError()

In [8]:
# generate testing cstr
isOverflow = True
while isOverflow == True:
    try:
        x_test_cstr, y_test_cstr = gen_cstr(F_cstr, V_cstr, C_A0s_cstr, k_0_cstr, E_cstr, R_cstr, T_0_cstr, delta_H_cstr, rho_L_cstr, C_p_cstr, Q_s_cstr, t_final_cstr, t_step_cstr, num_step, num_dims)
        isOverflow = False
    except ValueError:
        pass

sample_idx = rng.choice(len(x_test_cstr), size=1)
xtest_plot_1shot_cstr = x_test_cstr[sample_idx]
ytest_plot_1shot_cstr = y_test_cstr[sample_idx]

sample_idx = rng.choice(len(x_test_cstr), size=5)
xtest_plot_5shot_cstr = x_test_cstr[sample_idx]
ytest_plot_5shot_cstr = y_test_cstr[sample_idx]

sample_idx = rng.choice(len(x_test_cstr), size=10)
xtest_plot_10shot_cstr = x_test_cstr[sample_idx]
ytest_plot_10shot_cstr = y_test_cstr[sample_idx]

# generate testing batch
isOverflow = True
while isOverflow == True:
    try:
        x_test_batch, y_test_batch = gen_batch(V_batch, k_0_batch, E_batch, R_batch, delta_H_batch, rho_L_batch, C_p_batch, Q_s_batch, t_final_batch, t_step_batch, num_step, num_dims)
        isOverflow = False
    except ValueError:
        pass

sample_idx = rng.choice(len(x_test_batch), size=1)
xtest_plot_1shot_batch = x_test_batch[sample_idx]
ytest_plot_1shot_batch = y_test_batch[sample_idx]

sample_idx = rng.choice(len(x_test_batch), size=5)
xtest_plot_5shot_batch = x_test_batch[sample_idx]
ytest_plot_5shot_batch = y_test_batch[sample_idx]

sample_idx = rng.choice(len(x_test_batch), size=10)
xtest_plot_10shot_batch = x_test_batch[sample_idx]
ytest_plot_10shot_batch = y_test_batch[sample_idx]

# generate testing pfr
isOverflow = True
while isOverflow == True:
    try:
        x_test_pfr, y_test_pfr = gen_pfr(u_PFR, delH_term_PFR, k_0_PFR, C_p_PFR, rho_L_PFR, E_by_R_PFR, U_PFR, At_PFR, A_PFR, Tc_s_PFR, length_PFR, N_PFR, t_final_PFR, t_step_PFR, num_step, num_dims)
        isOverflow = False
    except ValueError:
        pass

sample_idx = rng.choice(len(x_test_pfr), size=1)
xtest_plot_1shot_pfr = x_test_pfr[sample_idx]
ytest_plot_1shot_pfr = y_test_pfr[sample_idx]

sample_idx = rng.choice(len(x_test_pfr), size=5)
xtest_plot_5shot_pfr = x_test_pfr[sample_idx]
ytest_plot_5shot_pfr = y_test_pfr[sample_idx]

sample_idx = rng.choice(len(x_test_pfr), size=10)
xtest_plot_10shot_pfr = x_test_pfr[sample_idx]
ytest_plot_10shot_pfr = y_test_pfr[sample_idx]

Number of training samples of CSTR:  6320
Number of training samples of Batch:  6019
Number of training samples of PFR:  6400


In [9]:
training_loss = np.zeros(niterations)
test_loss_1shot_cstr = np.zeros(int(niterations/eval_step+1))
test_loss_5shot_cstr = np.zeros(int(niterations/eval_step+1))
test_loss_10shot_cstr = np.zeros(int(niterations/eval_step+1))
test_loss_1shot_batch = np.zeros(int(niterations/eval_step+1))
test_loss_5shot_batch = np.zeros(int(niterations/eval_step+1))
test_loss_10shot_batch = np.zeros(int(niterations/eval_step+1))
test_loss_1shot_pfr = np.zeros(int(niterations/eval_step+1))
test_loss_5shot_pfr = np.zeros(int(niterations/eval_step+1))
test_loss_10shot_pfr = np.zeros(int(niterations/eval_step+1))

reactor = 0
count = 0

# reptile
for iteration in range(niterations):

    print("Begin iteration ", iteration)

    # generate task
    isOverflow = True
    while isOverflow == True:
        try:
            # switch to another reactor, 0 for cstr, 1 for batch, 2 for pfr
            if reactor == 0:
                x_all, y_all = gen_cstr(F_cstr, V_cstr, C_A0s_cstr, k_0_cstr, E_cstr, R_cstr, T_0_cstr, delta_H_cstr, rho_L_cstr, C_p_cstr, Q_s_cstr, t_final_cstr, t_step_cstr, num_step, num_dims)
                reactor = 1
            elif reactor == 1:
                x_all, y_all = gen_batch(V_batch, k_0_batch, E_batch, R_batch, delta_H_batch, rho_L_batch, C_p_batch, Q_s_batch, t_final_batch, t_step_batch, num_step, num_dims)
                reactor = 2
            elif reactor == 2:
                x_all, y_all = gen_pfr(u_PFR, delH_term_PFR, k_0_PFR, C_p_PFR, rho_L_PFR, E_by_R_PFR, U_PFR, At_PFR, A_PFR, Tc_s_PFR, length_PFR, N_PFR, t_final_PFR, t_step_PFR, num_step, num_dims)
                reactor = 0
            isOverflow = False
        except ValueError:
            pass

    inds = rng.permutation(len(x_all))

    # for some k number of iterations perform optimization on the task
    for k in range(innerepochs):

        for start in range(0, len(x_all), ntrain):
            mbinds = inds[start:start+ntrain]
            train_on_batch(x_all[mbinds], y_all[mbinds], model, optimizer)

    training_loss[iteration] = compute_loss(x_all, y_all, model)
    print("Training loss: ", training_loss[iteration])

    # reset weights and terminate for NaN training loss
    if np.isnan(training_loss[iteration]) == True:
        model.set_weights(weights_before)
        break

    # begin evaluation
    if plot and iteration==0 or (iteration+1) % eval_step == 0:
        # 1 shot learning for cstr
        weights_before = deepcopy(model.get_weights())
        for inneriter in range(50):
            train_on_batch(xtest_plot_1shot_cstr, ytest_plot_1shot_cstr, model, optimizer)

        test_loss_1shot_cstr[count] = compute_loss(x_test_cstr, y_test_cstr, model)
        print("Test loss for 1-shot CSTR: ", test_loss_1shot_cstr[count])
        model.set_weights(weights_before)

        # 5 shot learning for cstr
        weights_before = deepcopy(model.get_weights())
        for inneriter in range(50):
            train_on_batch(xtest_plot_5shot_cstr, ytest_plot_5shot_cstr, model, optimizer)

        test_loss_5shot_cstr[count] = compute_loss(x_test_cstr, y_test_cstr, model)
        print("Test loss for 5-shots CSTR: ", test_loss_5shot_cstr[count])
        model.set_weights(weights_before)

        # 10 shot learning for cstr
        weights_before = deepcopy(model.get_weights())
        for inneriter in range(50):
            train_on_batch(xtest_plot_10shot_cstr, ytest_plot_10shot_cstr, model, optimizer)

        test_loss_10shot_cstr[count] = compute_loss(x_test_cstr, y_test_cstr, model)
        print("Test loss for 10-shots CSTR: ", test_loss_10shot_cstr[count])
        model.set_weights(weights_before)

        # 1 shot learning for batch
        weights_before = deepcopy(model.get_weights())
        for inneriter in range(50):
            train_on_batch(xtest_plot_1shot_batch, ytest_plot_1shot_batch, model, optimizer)

        test_loss_1shot_batch[count] = compute_loss(x_test_batch, y_test_batch, model)
        print("Test loss for 1-shot Batch: ", test_loss_1shot_batch[count])
        model.set_weights(weights_before)

        # 5 shot learning for bacth
        weights_before = deepcopy(model.get_weights())
        for inneriter in range(50):
            train_on_batch(xtest_plot_5shot_batch, ytest_plot_5shot_batch, model, optimizer)

        test_loss_5shot_batch[count] = compute_loss(x_test_batch, y_test_batch, model)
        print("Test loss for 5-shots Batch: ", test_loss_5shot_batch[count])
        model.set_weights(weights_before)

        # 10 shot learning for batch
        weights_before = deepcopy(model.get_weights())
        for inneriter in range(50):
            train_on_batch(xtest_plot_10shot_batch, ytest_plot_10shot_batch, model, optimizer)

        test_loss_10shot_batch[count] = compute_loss(x_test_batch, y_test_batch, model)
        print("Test loss for 10-shots Batch: ", test_loss_10shot_batch[count])
        model.set_weights(weights_before)

        # 1 shot learning for pfr
        weights_before = deepcopy(model.get_weights())
        for inneriter in range(50):
            train_on_batch(xtest_plot_1shot_pfr, ytest_plot_1shot_pfr, model, optimizer)

        test_loss_1shot_pfr[count] = compute_loss(x_test_pfr, y_test_pfr, model)
        print("Test loss for 1-shot PFR: ", test_loss_1shot_pfr[count])
        model.set_weights(weights_before)

        # 5 shot learning for pfr
        weights_before = deepcopy(model.get_weights())
        for inneriter in range(50):
            train_on_batch(xtest_plot_5shot_pfr, ytest_plot_5shot_pfr, model, optimizer)

        test_loss_5shot_pfr[count] = compute_loss(x_test_pfr, y_test_pfr, model)
        print("Test loss for 5-shots PFR: ", test_loss_5shot_pfr[count])
        model.set_weights(weights_before)

        # 10 shot learning for pfr
        weights_before = deepcopy(model.get_weights())
        for inneriter in range(50):
            train_on_batch(xtest_plot_10shot_pfr, ytest_plot_10shot_pfr, model, optimizer)

        test_loss_10shot_pfr[count] = compute_loss(x_test_pfr, y_test_pfr, model)
        print("Test loss for 10-shots PFR: ", test_loss_10shot_pfr[count])
        model.set_weights(weights_before)

        count = count + 1

    if (iteration+1) % 100 == 0:
        filename = 'model_transfer_cstr_batch_pfr.sav'
        pickle.dump(model, open(filename, 'wb'))

Begin iteration  0
Number of training samples of CSTR:  6400
Training loss:  0.016093988294526258
Test loss for 1-shot CSTR:  0.08086334544218457
Test loss for 5-shots CSTR:  0.059000803224904905
Test loss for 10-shots CSTR:  0.03422383396061274
Test loss for 1-shot Batch:  2.3419961297236154
Test loss for 5-shots Batch:  0.8658415528885363
Test loss for 10-shots Batch:  0.8744941439626179
Test loss for 1-shot PFR:  0.5130990426934245
Test loss for 5-shots PFR:  0.2578239282073535
Test loss for 10-shots PFR:  0.4687442444663829
Begin iteration  1
Number of training samples of Batch:  6240
Training loss:  0.03920148058549375
Begin iteration  2
Number of training samples of PFR:  6400
Training loss:  0.00876241863808824
Begin iteration  3
Number of training samples of CSTR:  6233
Training loss:  0.013237124368946961
Begin iteration  4
Number of training samples of Batch:  6214
Training loss:  0.0890610388245484
Begin iteration  5
Number of training samples of PFR:  6400
Training loss:  0

Training loss:  0.015189616126941976
Begin iteration  75
Number of training samples of CSTR:  6217
Training loss:  0.014024856628864305
Begin iteration  76
Number of training samples of Batch:  6240
Training loss:  0.02903554000909446
Begin iteration  77
Number of training samples of PFR:  6400
Training loss:  0.014187566096867853
Begin iteration  78
Number of training samples of CSTR:  6259
Training loss:  0.010786298367030048
Begin iteration  79
Number of training samples of Batch:  6240
Training loss:  0.03265482074363555
Begin iteration  80
Number of training samples of PFR:  6400
Training loss:  0.012302554288273019
Begin iteration  81
Number of training samples of CSTR:  6236
Training loss:  0.01079475350452332
Begin iteration  82
Number of training samples of Batch:  6240
Training loss:  0.016452499729532413
Begin iteration  83
Number of training samples of PFR:  6400
Training loss:  0.015431807731782248
Begin iteration  84
Number of training samples of CSTR:  6320
Training loss

Test loss for 1-shot PFR:  0.1630135680957817
Test loss for 5-shots PFR:  0.10310186134056364
Test loss for 10-shots PFR:  0.14355194437839422
Begin iteration  150
Number of training samples of CSTR:  6240
Training loss:  0.02553353485437988
Begin iteration  151
Number of training samples of Batch:  6240
Training loss:  0.05081695869801736
Begin iteration  152
Number of training samples of PFR:  6400
Training loss:  0.026941010826039526
Begin iteration  153
Number of training samples of CSTR:  6400
Training loss:  0.019561369055623976
Begin iteration  154
Number of training samples of Batch:  6240
Training loss:  0.043513758120298736
Begin iteration  155
Number of training samples of PFR:  6400
Training loss:  0.04265791058413479
Begin iteration  156
Number of training samples of CSTR:  6400
Training loss:  0.013872113295094184
Begin iteration  157
Number of training samples of Batch:  6240
Training loss:  0.04612112785505132
Begin iteration  158
Number of training samples of PFR:  640

Training loss:  0.03982202381057992
Begin iteration  227
Number of training samples of PFR:  6400
Training loss:  0.0094501061315336
Begin iteration  228
Number of training samples of CSTR:  6230
Training loss:  0.012033643243790654
Begin iteration  229
Number of training samples of Batch:  6240
Training loss:  0.03434857843282553
Begin iteration  230
Number of training samples of PFR:  6400
Training loss:  0.011414426102670547
Begin iteration  231
Number of training samples of CSTR:  6400
Training loss:  0.011762139896217796
Begin iteration  232
Number of training samples of Batch:  6200
Training loss:  0.04316167574474737
Begin iteration  233
Number of training samples of PFR:  6400
Training loss:  0.017358584325212395
Begin iteration  234
Number of training samples of CSTR:  6188
Training loss:  0.014166947172875
Begin iteration  235
Number of training samples of Batch:  6240
Training loss:  0.04484405544048974
Begin iteration  236
Number of training samples of PFR:  6400
Training l

Number of training samples of CSTR:  6072
Training loss:  0.021524842359717706
Begin iteration  301
Number of training samples of Batch:  6240
Training loss:  0.11122756222499708
Begin iteration  302
Number of training samples of PFR:  6400
Training loss:  0.012665745747294715
Begin iteration  303
Number of training samples of CSTR:  6080
Training loss:  0.018647244073315833
Begin iteration  304
Number of training samples of Batch:  6240
Training loss:  0.048165259606015455
Begin iteration  305
Number of training samples of PFR:  6400
Training loss:  0.017910956357986334
Begin iteration  306
Number of training samples of CSTR:  6264
Training loss:  0.014465048679381842
Begin iteration  307
Number of training samples of Batch:  6198
Training loss:  0.06357329106749669
Begin iteration  308
Number of training samples of PFR:  6400
Training loss:  0.024157675637522812
Begin iteration  309
Number of training samples of CSTR:  6306
Training loss:  0.013375235718371183
Begin iteration  310
Nu

...

Number of training samples of PFR:  6400
Training loss:  0.07702353049973786
Begin iteration  1140
Number of training samples of CSTR:  6266
Training loss:  0.02906614201256921
Begin iteration  1141
Number of training samples of Batch:  6240
Training loss:  0.06435391682970706
Begin iteration  1142
Number of training samples of PFR:  6400
Training loss:  0.054455040482943765
Begin iteration  1143
Number of training samples of CSTR:  6064
Training loss:  0.02831245797115644
Begin iteration  1144
Number of training samples of Batch:  6240
Training loss:  0.03852310694903745
Begin iteration  1145
Number of training samples of PFR:  6400
Training loss:  0.023989951137996207
Begin iteration  1146
Number of training samples of CSTR:  6400
Training loss:  0.011908329027138773
Begin iteration  1147
Number of training samples of Batch:  6240
Training loss:  0.03447167611381812
Begin iteration  1148
Number of training samples of PFR:  6400
Training loss:  0.016669099408060453
Begin iteration  11

Training loss:  0.03201143390548392
Begin iteration  1213
Number of training samples of Batch:  6235
Training loss:  0.1326479801398874
Begin iteration  1214
Number of training samples of PFR:  6400
Training loss:  0.07896239598961556
Begin iteration  1215
Number of training samples of CSTR:  6343
Training loss:  0.049904785173696724
Begin iteration  1216
Number of training samples of Batch:  6240
Training loss:  0.14136581402819498
Begin iteration  1217
Number of training samples of PFR:  6400
Training loss:  0.03834535694490222
Begin iteration  1218
Number of training samples of CSTR:  6080
Training loss:  0.025981882205164678
Begin iteration  1219
Number of training samples of Batch:  6240
Training loss:  0.10837158919978566
Begin iteration  1220
Number of training samples of PFR:  6400
Training loss:  0.024446013374008216
Begin iteration  1221
Number of training samples of CSTR:  6320
Training loss:  0.018825994291641154
Begin iteration  1222
Number of training samples of Batch:  6

Training loss:  0.04449825910833411
Begin iteration  1291
Number of training samples of Batch:  6240
Training loss:  0.11334364487396795
Begin iteration  1292
Number of training samples of PFR:  6400
Training loss:  0.05332700506002137
Begin iteration  1293
Number of training samples of CSTR:  6400
Training loss:  0.03560987301185177
Begin iteration  1294
Number of training samples of Batch:  6023
Training loss:  0.04821864565699323
Begin iteration  1295
Number of training samples of PFR:  6400
Training loss:  0.03616938440585501
Begin iteration  1296
Number of training samples of CSTR:  6240
Training loss:  0.03010506679447308
Begin iteration  1297
Number of training samples of Batch:  6240
Training loss:  0.0735960964724544
Begin iteration  1298
Number of training samples of PFR:  6400
Training loss:  0.02340544295052404
Begin iteration  1299
Number of training samples of CSTR:  6265
Training loss:  0.026648187814221964
Test loss for 1-shot CSTR:  0.13897239633842112
Test loss for 5-

Number of training samples of PFR:  6400
Training loss:  0.06514515243832122
Begin iteration  1365
Number of training samples of CSTR:  6400
Training loss:  0.04047635447478946
Begin iteration  1366
Number of training samples of Batch:  6240
Training loss:  0.05834126871931808
Begin iteration  1367
Number of training samples of PFR:  6400
Training loss:  0.02238479636641794
Begin iteration  1368
Number of training samples of CSTR:  6320
Training loss:  0.024050551125816985
Begin iteration  1369
Number of training samples of Batch:  6240
Training loss:  0.08027087629319318
Begin iteration  1370
Number of training samples of PFR:  6400
Training loss:  0.14364595780439307
Begin iteration  1371
Number of training samples of CSTR:  6228
Training loss:  0.029257503439713697
Begin iteration  1372
Number of training samples of Batch:  6240
Training loss:  0.07313912820343921
Begin iteration  1373
Number of training samples of PFR:  6400
Training loss:  0.16024354115925768
Begin iteration  1374

Number of training samples of PFR:  6400
Training loss:  0.035732597909389444
Begin iteration  1443
Number of training samples of CSTR:  6155
Training loss:  0.018702480552440774
Begin iteration  1444
Number of training samples of Batch:  6238
Training loss:  0.07323904152443199
Begin iteration  1445
Number of training samples of PFR:  6400
Training loss:  0.03484529728564209
Begin iteration  1446
Number of training samples of CSTR:  6400
Training loss:  0.025943954123427048
Begin iteration  1447
Number of training samples of Batch:  6240
Training loss:  0.04105228800305632
Begin iteration  1448
Number of training samples of PFR:  6400
Training loss:  0.05758824618745272
Begin iteration  1449
Number of training samples of CSTR:  6110
Training loss:  0.04836527860151888
Test loss for 1-shot CSTR:  0.10348793348581523
Test loss for 5-shots CSTR:  0.08534720201290469
Test loss for 10-shots CSTR:  0.03830014351036778
Test loss for 1-shot Batch:  2.356032842765435
Test loss for 5-shots Batc

In [10]:
np.savetxt("training_transfer_cstr_batch_pfr.txt", training_loss, fmt='%f', delimiter=" ")
np.savetxt("testing_1shot_transfer_cstr_cstr_batch_pfr.txt", test_loss_1shot_cstr, fmt='%f', delimiter=" ")
np.savetxt("testing_5shot_transfer_cstr_cstr_batch_pfr.txt", test_loss_5shot_cstr, fmt='%f', delimiter=" ")
np.savetxt("testing_10shot_transfer_cstr_cstr_batch_pfr.txt", test_loss_10shot_cstr, fmt='%f', delimiter=" ")
np.savetxt("testing_1shot_transfer_batch_cstr_batch_pfr.txt", test_loss_1shot_batch, fmt='%f', delimiter=" ")
np.savetxt("testing_5shot_transfer_batch_cstr_batch_pfr.txt", test_loss_5shot_batch, fmt='%f', delimiter=" ")
np.savetxt("testing_10shot_transfer_batch_cstr_batch_pfr.txt", test_loss_10shot_batch, fmt='%f', delimiter=" ")
np.savetxt("testing_1shot_transfer_pfr_cstr_batch_pfr.txt", test_loss_1shot_pfr, fmt='%f', delimiter=" ")
np.savetxt("testing_5shot_transfer_pfr_cstr_batch_pfr.txt", test_loss_5shot_pfr, fmt='%f', delimiter=" ")
np.savetxt("testing_10shot_transfer_pfr_cstr_batch_pfr.txt", test_loss_10shot_pfr, fmt='%f', delimiter=" ")

In [11]:
filename = 'model_transfer_cstr_batch_pfr.sav'
pickle.dump(model, open(filename, 'wb'))
# model = pickle.load(open(filename, 'rb'))